In [7]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [11]:
dataset_path = '/content/drive/MyDrive/G3CNN1/dataset'  # Update this to your dataset folder


In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
import os


In [13]:
# Data generators for training and validation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # EfficientNet preprocessing
    validation_split=0.2,                     # 20% validation split
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Training generator
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),  # Resize images to 224x224 for EfficientNet
    batch_size=32,
    class_mode='binary',     # Binary classification
    subset='training'        # Use the training subset
)

# Validation generator
val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'      # Use the validation subset
)


Found 156 images belonging to 2 classes.
Found 38 images belonging to 2 classes.


In [14]:
# Load EfficientNetB0 base model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
base_model.trainable = False

# Add custom classification layers
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)          │ (None, 7, 7, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,213,668 (16.07 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [15]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  # Adjust based on dataset size and performance
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 112s 14s/step - accuracy: 0.4924 - loss: 0.8170 - val_accuracy: 0.6250 - val_loss: 0.5838
Epoch 2/10
1/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.7500 - loss: 0.5013

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 137ms/step - accuracy: 0.7500 - loss: 0.5013 - val_accuracy: 0.6667 - val_loss: 0.5084
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 42s 3s/step - accuracy: 0.6300 - loss: 0.7257
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 16s 5s/step - accuracy: 0.8125 - loss: 0.4583 - val_accuracy: 0.6562 - val_loss: 0.6185
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 64s 3s/step - accuracy: 0.7417 - loss: 0.5294 - val_accuracy: 0.8333 - val_loss: 0.4276
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.7500 - loss: 0.4450
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 95s 7s/step - accuracy: 0.7429 - loss: 0.5334 - val_accuracy: 0.7500 - val_loss: 0.4898
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 115ms/step - accuracy: 0.6875 - loss: 0.5757 - val_accuracy: 0.6667 - val_loss: 0.4980
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 66s 10s/step - accuracy: 0.8098 - loss: 0.4416
Epoch 10/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 15s 4s/step - accuracy: 0.8125 - loss: 0.5226 - val_accuracy: 0.6562 - val_loss: 0.6158


In [16]:
# Evaluate the model
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_acc:.2f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 14s 364ms/step - accuracy: 0.6502 - loss: 0.6352
Validation Accuracy: 0.63


In [17]:
# Save the model
model.save('/content/drive/MyDrive/efficientnetb0_iris_tumor_classifier.h5')
print("Model saved to Google Drive.")


Model saved to Google Drive.


In [21]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def predict_image(img_path, model):
    # Load and preprocess the image
    image = load_img(img_path, target_size=(224, 224))
    image_array = img_to_array(image)
    image_array = preprocess_input(image_array)  # Preprocess for EfficientNet
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension

    # Predict
    prediction = model.predict(image_array)[0][0]
    return "Tumor" if prediction > 0.5 else "Non-Tumor"

# Load the saved model
saved_model = tf.keras.models.load_model('/content/drive/MyDrive/efficientnetb0_iris_tumor_classifier.h5')

# Example prediction
test_image_path = '/content/drive/MyDrive/004_06_L.bmp'  # Update with your test image path
print("Prediction:", predict_image(test_image_path, saved_model))


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Prediction: Non-Tumor


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define paths
base_dir = "/content/drive/MyDrive/G3CNN1/dataset"  # Replace with your folder path
tumor_dir = os.path.join(base_dir, "tumor")
non_tumor_dir = os.path.join(base_dir, "no_tumor")

# Image parameters
IMG_SIZE = (224, 224)  # Input size for VGG16
BATCH_SIZE = 32

# Data generators for loading and augmenting images
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80-20 train-validation split
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

# Load VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
EPOCHS = 10
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS
)

# Save the trained model
model.save('/content/drive/MyDrive/tumor_vgg16_model.h5')

# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy:.2f}")

# Function to predict on new images
def predict_image(image_path):
    from tensorflow.keras.preprocessing.image import load_img, img_to_array

    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    prediction = model.predict(img_array)
    if prediction[0][0] > 0.5:
        print(f"Tumor detected with probability: {prediction[0][0]:.2f}")
    else:
        print(f"No tumor detected with probability: {1 - prediction[0][0]:.2f}")

# Example usage
test_image = "/content/drive/MyDrive/001_06_L.bmp"  # Replace with your test image
predict_image(test_image)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 156 images belonging to 2 classes.
Found 38 images belonging to 2 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 196s 30s/step - accuracy: 0.6084 - loss: 0.7875 - val_accuracy: 0.6842 - val_loss: 0.6382
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 160s 26s/step - accuracy: 0.6318 - loss: 0.6391 - val_accuracy: 0.6842 - val_loss: 0.5772
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 196s 25s/step - accuracy: 0.6763 - loss: 0.6556 - val_accuracy: 0.6579 - val_loss: 0.6162
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 219s 28s/step - accuracy: 0.7699 - loss: 0.4799 - val_accuracy: 0.7632 - val_loss: 0.5470
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 197s 29s/step - accuracy: 0.7990 - loss: 0.4670 - val_accuracy: 0.6316 - val_loss: 0.6288
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 203s 28s/step - accuracy: 0.7683 - loss: 0.4472 - val_accuracy: 0.7632 - val_loss: 0.5261
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 155s 27s/step - accuracy: 0.8571 - loss: 0.3727 - val_accuracy: 0.7368 - val_loss: 0.5430
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 148s 26s/step - accuracy: 0.8316 - loss: 0.4109 - val_accuracy: 0.7632 - val_loss: 0.4853
Epo

2/2 ━━━━━━━━━━━━━━━━━━━━ 31s 4s/step - accuracy: 0.6749 - loss: 0.5401
Validation Accuracy: 0.68
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
No tumor detected with probability: 0.96


In [8]:
# Import necessary libraries
import gradio as gr
import numpy as np
import matplotlib.pyplot as plt

# Define a function to classify the image
def classify_image(image):
    # Dummy model prediction: replace with actual model
    # For demonstration, we randomly decide if a tumor is detected
    if np.random.rand() > 0.5:
        prediction = "Tumor Detected"
        probability = np.random.rand()
    else:
        prediction = "No Tumor"
        probability = 1 - np.random.rand()

    # Return the prediction and probability
    return f"{prediction} (Probability: {probability:.2f})"

# Create a Gradio interface
demo = gr.Interface(
    fn=classify_image,  # Function to classify the image
    inputs=gr.Image(type="numpy"),  # Input component for image upload
    outputs=gr.Textbox(label="Prediction"),  # Output component for prediction
    title="Tumor Detection",
    description="Upload an image to detect the presence of a tumor. The model will analyze the image and classify it as either 'Tumor Detected' or 'No Tumor', along with the probability of the prediction."
)

# Launch the interface
if __name__ == "__main__":
    demo.launch(show_error=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://192d4afe392d5ddc6b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
